In [1]:
from astropy.table import Table, Column
import time
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import sys
from scipy.stats import kde
from tqdm import tqdm

sys.path.insert(0, '../Modules')
import erqScaling
import erqMedSpec
import os

In [1]:
range(40, 100, 10)

range(40, 100, 10)

In [2]:
_[-1]

90

In [2]:
# Reading data
smp=Table.read('../LumMatch/LumMatch.fits')
iW3_0 = smp['i-w3']
kt80_0= smp['kurt80_gf']
rew_0 = smp['rew_gf']
rew_0  = np.log10(rew_0)
frat_0 = smp['frat_nv/civ']
fwhm_0 = smp['fwhm_gf']
rw3_0=smp['r-w3']
rz_0=smp['r-z']
iz_0 = iW3_0 - rw3_0 + rz_0
Lum_0 = smp['Lum']
z_dr12 = smp['z_dr12']
plate = smp['Plate']
mjd = smp['MJD']
fiber =  smp['FiberID']

In [3]:
# Normalizing data + getting the center and ERQ
data_0=np.array(list(zip(iW3_0, rew_0)))
(data, minData, rangeData) =  erqScaling.MinMaxScaler(data_0)
Main_center = np.median(data, axis=0)
ERQ = data[((iW3_0>=4.6) & (rew_0>=2))]


enclosing_ratio=1; resolution=1000; ngrids=500; 
opening_angle, ERQ_direction = erqMedSpec.opening_angle_finder(ERQ, Main_center, enclosing_ratio, resolution)
#  Obtaining Point A and B for the ERQ wedge
x_erq, y_erq= np.median(ERQ, axis=0)  #center of ERQ population 
r_erq = np.sqrt((x_erq-Main_center[0])**2 + (y_erq-Main_center[1])**2)  # ERQ radius 

#  Obtaining Point A and B for the ERQ wedge
x_erq, y_erq= np.median(ERQ, axis=0)  #center of ERQ population 
r_erq = np.sqrt((x_erq-Main_center[0])**2 + (y_erq-Main_center[1])**2)  # ERQ radius 

In [4]:
# binning wedge-1
A= [0,0]
B= [0,0]
ext=1.6
A[0] = Main_center[0] + r_erq*ext*np.cos(ERQ_direction+opening_angle)
B[0] = Main_center[0] + r_erq*ext*np.cos(ERQ_direction-opening_angle)
A[1] = Main_center[1] + r_erq*ext*np.sin(ERQ_direction+opening_angle)
B[1] = Main_center[1] + r_erq*ext*np.sin(ERQ_direction-opening_angle)
np.rad2deg(opening_angle)

bw=3 # bandwidth ratio for the default value
wdg=0
dir = 'outData/'
x_erq_0 = 4.6
y_erq_0 = 2
if os.path.exists(dir)==False:
    os.system('mkdir -p ' + dir)
levels = np.array([0.03, .1, 0.3])

In [22]:
from sklearn.neighbors import LocalOutlierFactor as LOF

nBin=7
expansion=[1.26, 1.54, 1.95]
bin_label, bin_pop = erqMedSpec.KDE_Bin2D(data, rangeData, minData, False, 4.6, 2,
                                          50,  bw, levels, A, B, True,expansion,'', False, '','' ,'', False)
 

for k in [38,39,40, 41]:
    clf= LOF(n_neighbors=k,  n_jobs=-1)
    clf.fit_predict(data)
    LOFScores= -clf.negative_outlier_factor_
    LOFScoresMed=np.zeros([nBin])
    #  Median properties in each bin
    for b in range(nBin):
        mask = (bin_label==b+1)
        LOFScoresMed[b] =np.median(LOFScores[mask])
    
    max_diff= -100
    if ((LOFScoresMed[3]-LOFScoresMed[4])>max_diff):
        max_diff = LOFScoresMed[3]-LOFScoresMed[4]
        min_LOF_B4 = LOFScoresMed[3]
        max_LOF_B5 = LOFScoresMed[4]
        i_min=i
        j_min=j
    print(k, max_diff)
    plt.plot(np.linspace(1,nBin,nBin), LOFScoresMed, marker='o', label='k:%d'%(k), alpha=0.5)
plt.legend()

In [5]:
from matplotlib import path
p = path.Path([(0,0), (0, 1), (1, 1), (1, 0)])  # square with legs length 1 and bottom left corner at the origin
p.contains_points([[0.5,0.5], [0,2]])
cc=[1,3]
bb=[4,5]
ff=[6,8]
p=path.Path([cc,bb,ff])
p.contains_points([[3,3], [4,6]])

array([False,  True])

In [12]:
j_min

-1

In [8]:
min_LOF_B4

1.0630475225943228

In [9]:
max_LOF_B5

1.0562629337443807